### Import der Bibliotheken



In [ ]:
# Importieren der notwendigen Bibliotheken
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands
import warnings

warnings.filterwarnings("ignore")


### Top 10 Coins nach Marketcap

In [ ]:
def fetch_top_10_coins():
    """Fetches the top 10 coins by market cap from CoinGecko and excludes blacklisted pairs."""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 10,
        "page": 1,
        "sparkline": False
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        
        # Blacklist for unwanted pairs
        blacklist = ["USDT", "STETH"]
        
        # Filter coins: exclude symbols in blacklist
        coins = [
            {"Symbol": coin["symbol"].upper(), "Name": coin["name"], "Market Cap": coin["market_cap"]}
            for coin in data if coin["symbol"].upper() not in blacklist
        ]
        return pd.DataFrame(coins)
    else:
        print("Fehler beim Abrufen der Daten:", response.status_code)
        return None


### Historische daten - Abrufen der Daten

In [ ]:
def fetch_historical_data_complete(symbol, interval="15m", days=180):
    """Fetch historical crypto data from Binance."""
     # Blacklist unwanted pairs
    blacklist = ["USDTUSDT", "STETHUSDT"]
    if symbol.upper() in blacklist:
        print(f"Symbol {symbol} ist geblacklistet. Überspringe...")
        return None
    base_url = "https://api.binance.com/api/v3/klines"
    end_time = int(datetime.now().timestamp() * 1000)  # Current time in ms
    start_time = int((datetime.now() - timedelta(days=days)).timestamp() * 1000)  # Start time in ms

    all_data = []
    while start_time < end_time:
        params = {
            "symbol": symbol.upper(),
            "interval": interval,
            "startTime": start_time,
            "endTime": end_time,
            "limit": 1000,
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if not data:
                break
            all_data.extend(data)
            start_time = data[-1][6]
        else:
            print(f"Error fetching data for {symbol}: {response.status_code}")
            break

    df = pd.DataFrame(all_data, columns=[
        "Open Time", "Open", "High", "Low", "Close", "Volume",
        "Close Time", "Quote Asset Volume", "Number of Trades",
        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
    ])
    df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
    df["Close Time"] = pd.to_datetime(df["Close Time"], unit="ms")

    return df[["Open Time", "Open", "Close", "High", "Low"]]


### Lokale Speicherung/Laden

In [ ]:
def fetch_or_load_historical_data(coin, interval="15m", days=180, save_path="historical_data"):
    """
    Fetches historical data for a coin from Binance or loads it from a local file.
    """
    os.makedirs(save_path, exist_ok=True)
    file_path = os.path.join(save_path, f"{coin}_{interval}.csv")

    # Prüfen, ob die Datei existiert
    if os.path.exists(file_path):
        print(f"Lade historische Daten für {coin} aus {file_path}...")
        return pd.read_csv(file_path, parse_dates=["Open Time"])

    # Abrufen der Daten von Binance, falls nicht lokal vorhanden
    print(f"Abrufen der historischen Daten für {coin} von Binance...")
    coin_data = fetch_historical_data_complete(coin, interval=interval, days=days)
    if coin_data is not None:
        coin_data.to_csv(file_path, index=False)
        print(f"Historische Daten für {coin} gespeichert in {file_path}.")
    return coin_data


### Berechnung der technischen Indikatoren

In [ ]:
def calculate_indicators(coin_data):
    """Calculate RSI, MACD, and Bollinger Bands."""
    for column in ["Open", "Close", "High", "Low"]:
        coin_data[column] = pd.to_numeric(coin_data[column], errors="coerce")
    
    # RSI
    coin_data["RSI"] = RSIIndicator(close=coin_data["Close"], window=14).rsi()
    
    # MACD
    macd = MACD(close=coin_data["Close"])
    coin_data["MACD"] = macd.macd()
    coin_data["Signal"] = macd.macd_signal()
    
    # Bollinger Bands
    bb = BollingerBands(close=coin_data["Close"])
    coin_data["BB Upper"] = bb.bollinger_hband()
    coin_data["BB Lower"] = bb.bollinger_lband()
    
    return coin_data


### Backtesting-Logik

In [ ]:
def simulate_long_trades_debug(coin_data, rsi_threshold=30, take_profit=0.02, stop_loss=0.01, debug=True):
    """
    Simulates long trades with improved entry/exit conditions and debugging.

    Parameters:
        coin_data (DataFrame): The historical data with indicators.
        rsi_threshold (float): RSI value below which to enter a trade.
        take_profit (float): Take profit level as a percentage.
        stop_loss (float): Stop loss level as a percentage.
        debug (bool): If True, prints debug information.

    Returns:
        DataFrame: Trade details with entry and exit points.
    """
    trades = []
    position_open = False
    entry_price = 0
    highest_price = 0

    for i in range(len(coin_data)):
        row = coin_data.iloc[i]
        close_price = row["Close"]

        # Debugging: Aktuelle Werte ausgeben
        if debug:
            print(f"Time: {row['Open Time']}, Close: {close_price:.2f}, RSI: {row['RSI']:.2f}, MACD: {row['MACD']:.2f}, Signal: {row['Signal']:.2f}")

        # Einstiegskriterien
        if not position_open:
            if (
                row["RSI"] < rsi_threshold and
                row["MACD"] > row["Signal"] and
                close_price < row["BB Lower"]
            ):
                position_open = True
                entry_price = close_price
                highest_price = close_price
                trades.append({
                    "Type": "Buy",
                    "Price": entry_price,
                    "Time": row["Open Time"]
                })
                if debug:
                    print(f"Buy at {entry_price:.2f} on {row['Open Time']}")

        # Ausstiegskriterien
        elif position_open:
            profit = (close_price - entry_price) / entry_price

            # Trailing Stop-Loss
            highest_price = max(highest_price, close_price)
            trailing_exit_price = highest_price * (1 - stop_loss)
            if close_price <= trailing_exit_price:
                position_open = False
                trades.append({
                    "Type": "Sell",
                    "Price": close_price,
                    "Time": row["Open Time"]
                })
                if debug:
                    print(f"Sell (Trailing Stop) at {close_price:.2f} on {row['Open Time']}")
                continue

            # Take-Profit oder Stop-Loss
            if profit >= take_profit or profit <= -stop_loss:
                position_open = False
                trades.append({
                    "Type": "Sell",
                    "Price": close_price,
                    "Time": row["Open Time"]
                })
                if debug:
                    print(f"Sell (TP/SL) at {close_price:.2f} on {row['Open Time']}")

    return pd.DataFrame(trades)


### Berechnung der Performance

In [ ]:
def calculate_performance(trades, initial_balance=1000):
    """
    Calculates the performance of the backtesting.

    Parameters:
        trades (DataFrame): The trade details.
        initial_balance (float): Starting capital.

    Returns:
        dict: Performance metrics (e.g., Net Profit, Winrate).
    """
    balance = initial_balance
    wins = 0
    losses = 0

    for i in range(0, len(trades), 2):  # Iteriere über Buy-Sell-Paare
        if i + 1 < len(trades):
            entry = trades.iloc[i]["Price"]
            exit = trades.iloc[i + 1]["Price"]
            profit = (exit - entry) / entry
            balance += balance * profit

            if profit > 0:
                wins += 1
            else:
                losses += 1

    winrate = wins / (wins + losses) if (wins + losses) > 0 else 0

    return {
        "Net Profit": balance - initial_balance,
        "Winrate": winrate,
        "Trades": len(trades) // 2
    }


### Visualisierung der Kapitalkurve

In [ ]:
def plot_equity_curve(trades, initial_balance=1000):
    """
    Plots the equity curve over the trades.

    Parameters:
        trades (DataFrame): The trade details.
        initial_balance (float): Starting capital.
    """
    balance = initial_balance
    equity_curve = [balance]

    for i in range(0, len(trades), 2):  # Iteriere über Buy-Sell-Paare
        if i + 1 < len(trades):
            entry = trades.iloc[i]["Price"]
            exit = trades.iloc[i + 1]["Price"]
            profit = (exit - entry) / entry
            balance += balance * profit
            equity_curve.append(balance)

    plt.figure(figsize=(12, 6))
    plt.plot(equity_curve, label="Equity Curve", color="blue", linewidth=2)
    plt.title("Equity Curve")
    plt.xlabel("Trades")
    plt.ylabel("Balance")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


### Optimierungmodul (Platzhalter)

In [ ]:
from itertools import product

def optimize_parameters(coin_data, sl_tp_ratio=1.5):
    """
    Optimizes trading parameters for a given coin's data.

    Parameters:
        coin_data (DataFrame): Historical data with indicators.
        sl_tp_ratio (float): The minimum SL:TP ratio for a profitable trade.

    Returns:
        dict: Best parameters and their performance.
    """
    # Eingrenzung der Parameterbereiche
    rsi_values = range(30, 40, 5)  # Engerer RSI-Bereich
    tp_values = [0.02, 0.03]  # Begrenzte TP-Werte
    sl_values = [tp / sl_tp_ratio for tp in tp_values]  # SL basierend auf SL:TP-Verhältnis
    macd_thresholds = [0]  # Einfachheitshalber nur MACD > Signal

    best_params = None
    best_performance = -float("inf")

    # Alle Parameterkombinationen testen
    for rsi, tp, sl, macd in product(rsi_values, tp_values, sl_values, macd_thresholds):
        # Backtesting mit aktuellen Parametern
        trades = simulate_long_trades_debug(coin_data, rsi_threshold=rsi, take_profit=tp, stop_loss=sl)
        
        # Performance berechnen
        performance = calculate_performance(trades)

        # Beste Parameter aktualisieren
        if performance["Net Profit"] > best_performance:
            best_performance = performance["Net Profit"]
            best_params = {
                "RSI": rsi,
                "Take Profit": tp,
                "Stop Loss": sl,
                "MACD Threshold": macd,
                "Net Profit": best_performance,
                "Winrate": performance["Winrate"],
            }

    return best_params


### Visualisierung

In [ ]:
def plot_trades(coin_name, coin_data, trades):
    """Plot price data and trades with indicators."""
    plt.figure(figsize=(14, 8))
    plt.plot(coin_data["Open Time"], coin_data["Close"], label="Close Price", color="blue", linewidth=1.5)
    
    # Überprüfen, ob Trades korrekt sind
    if not trades.empty and "Time" in trades.columns and "Price" in trades.columns:
        plt.scatter(trades["Time"], trades["Price"], color="red", label="Trades")
    else:
        print(f"Warnung: Keine gültigen Trades für {coin_name}.")
    
    plt.title(f"{coin_name} Price with Trades")
    plt.xlabel("Time")
    plt.ylabel("Price (USD)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


### Pipeline für mehrere Coins

In [ ]:
# Abrufen der Top-10-Coins und Erstellen der Symbol-Liste
top_10_coins = fetch_top_10_coins()
coin_symbols = [f"{symbol}USDT" for symbol in top_10_coins["Symbol"]]
print("Top 10 Coin-Symbole:", coin_symbols)

# Optimierung der Parameter für jeden Coin
optimized_parameters = []

for coin in coin_symbols:
    print(f"Abrufen oder Laden der Daten für {coin}...")
    coin_data = fetch_or_load_historical_data(coin, interval="15m", days=180)
    if coin_data is not None:
        # Berechnung der Indikatoren
        coin_data = calculate_indicators(coin_data)

        # Optimierung der Parameter
        best_params = optimize_parameters(coin_data)
        optimized_parameters.append({"Coin": coin, **best_params})

        # Backtesting mit besten Parametern
        trades = simulate_long_trades_debug(
            coin_data,
            rsi_threshold=best_params["RSI"],
            take_profit=best_params["Take Profit"],
            stop_loss=best_params["Stop Loss"]
        )

        # Visualisierung der Trades
        plot_trades(coin, coin_data, trades)
    else:
        print(f"Keine Daten für {coin} verfügbar.")

# Speichern der optimierten Parameter
optimized_df = pd.DataFrame(optimized_parameters)
optimized_df.to_csv("optimized_parameters.csv", index=False)
print("Optimierte Parameter gespeichert in 'optimized_parameters.csv'.")
